## Packages

In [22]:
using NeuralOperators, CUDA, HDF5, Flux

## Define Device

In [23]:
if has_cuda()
    @info "Training on GPU"
    device = gpu
else
    @info "Training on CPU"
    device = cpu
end;

┌ Info: Training on GPU
└ @ Main c:\Users\kaoid\My Drive\Estudo\Poli\Disciplinas\PINNs\acousticPINNs\deepOnet_julia.ipynb:2


## Get data

In [24]:
dataset = []
h5open("./dataset.h5", "r") do f
    # read standardized [x y t p]ₙ data
    datasetName = "standardData"
    # or read unit normalized [x y t p]ₙ data
#     datasetName = "unitData"
    datasets = HDF5.get_datasets(f)
    for d in datasets
        if HDF5.name(d)[2 : end] == datasetName
            global dataset = HDF5.read(d)
        end
    end
end
dataset

391036×4 Matrix{Float64}:
 -1.61352  -1.84881   -1.71718  0.107731
 -1.56241  -1.84881   -1.71718  0.107731
 -1.51129  -1.84881   -1.71718  0.107731
 -1.46018  -1.84881   -1.71718  0.107731
 -1.40906  -1.84881   -1.71718  0.107731
 -1.35794  -1.84881   -1.71718  0.107731
 -1.30683  -1.84881   -1.71718  0.107731
 -1.25571  -1.84881   -1.71718  0.107731
 -1.2046   -1.84881   -1.71718  0.107731
 -1.15348  -1.84881   -1.71718  0.107731
  ⋮                             
  1.63928   0.712423   1.71718  0.0497845
  1.67994   0.712423   1.71718  0.0468581
  1.7206    0.712423   1.71718  0.045986
  1.76126   0.712423   1.71718  0.0475291
  1.80192   0.712423   1.71718  0.0500725
  1.84258   0.712423   1.71718  0.0519384
  1.88324   0.712423   1.71718  0.0528655
  1.9239    0.712423   1.71718  0.053352
  1.96456   0.712423   1.71718  0.0542554

## Organize data

In [25]:
# initial conditions u(x, 0)
initialTrain = dataset[1 : 3371, 4]' # there are 3371 nodes in the 2D FEA mesh
# solutions u(x, t_end)
endTrain = dataset[(end - 3370) : end, 4]
# values for validation
initialVal = dataset[(end - 300) : end, 4]' |> device
endVal = dataset[(end - 300) : end, 4] |> device
# 2D FEA mesh nodal positions
coords = dataset[1 : 3371, 1:2]' |> device
# training data
dataTrain = [(initialTrain, endTrain, coords)] |> device;

## Model definition

In [26]:
# architecture of branch network (functions)
branch = Chain(Dense(3371, 64, σ), Dense(64, 72, σ))
# architecture of trunk network (coordinates)
trunk = Chain(Dense(3371, 64, tanh), Dense(64, 72, tanh))
model = DeepONet(branch, trunk) |> device

DeepONet with
branch net: (Chain(Dense(3371 => 64, σ), Dense(64 => 72, σ)))
Trunk net: (Chain(Dense(3371 => 64, tanh), Dense(64 => 72, tanh)))


## Optimization

In [27]:
opt = Adam(1e-3) # optimizer
# loss definition
loss(X, y, sensor) = Flux.Losses.mse(model(X, sensor), y)
# progress callback
evalcb() = @show(loss(initialVal, endVal, coords))
# train the model
for epoch in 1:2
    Flux.train!(loss, Flux.params(model), dataTrain, opt, cb = evalcb)
end

DimensionMismatch: DimensionMismatch: layer Dense(3371 => 64, σ) expects size(input, 1) == 3371, but got 1×3371 adjoint(::CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}) with eltype Float32

## Model performance

In [28]:
# predictions for validation data
ỹ = m(initialVal |> device, coords |> device)
# absolute errors in validation
diffvec = vec(abs.(cpu(endVal) .- cpu(ỹ)))
# mean absolute error in validation
print(mean(diffvec))

UndefVarError: UndefVarError: m not defined